In [2]:
from normality import normalize
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from countrynames import to_code
import os
import shutil
import datetime


In [3]:



csv_to_fix = "edgar.csv"

now = datetime.datetime.now()
formatted_date = now.strftime("%Y%m%d")
folder_path = "%s_Country_Renamed_%s" %(csv_to_fix.split("_")[0], formatted_date)
os.makedirs(folder_path)
geo_df = pd.read_csv("./ISOToEcoInventToolSub/Geographies.csv")
df = pd.read_csv(csv_to_fix)



In [4]:
df['Region/Regional Scope'].unique()
# df['Country of Applicability'] = "GLO"
# df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)

array(['Malta', 'Myanmar', 'Mongolia', 'Mozambique', 'Mauritania',
       'Martinique', 'Mauritius', 'Malawi', 'Malaysia', 'Namibia',
       'New Caledonia', 'Niger', 'Guinea', 'Guadeloupe', 'Gambia',
       'Guinea-Bissau', 'Equatorial Guinea', 'Greece', 'Grenada',
       'Greenland', 'Guatemala', 'French Guiana', 'Guyana', 'Hong Kong',
       'Honduras', 'Seychelles', 'Syrian Arab Republic',
       'Turks and Caicos Islands', 'Chad', 'Togo', 'Thailand',
       'Tajikistan', 'Turkmenistan', 'Timor-Leste', 'Tonga',
       'Trinidad and Tobago', 'Tunisia', 'Nigeria', 'Nicaragua',
       'Netherlands', 'Sierra Leone', 'El Salvador', 'Somalia',
       'Saint Pierre and Miquelon', 'Sao Tome and Principe', 'Suriname',
       'Slovakia', 'Slovenia', 'Sweden', 'Swaziland', 'Papua New Guinea',
       'Poland', 'Puerto Rico', 'Portugal', 'Paraguay',
       'French Polynesia', 'Qatar', 'Reunion', 'Romania',
       'Russian Federation', 'Rwanda', 'Saudi Arabia',
       'Serbia and Montenegro', 'S

In [5]:
def normalize_name(country):
    """Clean up a country name before comparison."""
    return normalize(country, latinize=True)

df_ambig = pd.DataFrame(columns=["Input string", "Mapped name", "Mapped code", "Quarantine?"])

def mapping(country:str)->str:
    global df_ambig
    if(result:=to_code(country)) is not None:
        return result
    matching_rows = df_ambig[df_ambig["Input string"].str.contains(country, case=False, na=False)]
    if not matching_rows.empty:
        return country
    else:
        longname = process.extractOne(country, choices=geo_df['Name'], processor=(lambda x: str(x)), scorer=fuzz.token_set_ratio, score_cutoff=0)[0]
        shortname = geo_df[geo_df['Name'] == longname]['Shortname'].iloc[0]
        new_row = pd.DataFrame([{ 
            "Input string": country,
            "Mapped name": longname,
            "Mapped code": shortname}])
        df_ambig = pd.concat([df_ambig, new_row], ignore_index=True)
        return country
    
Region_column_name = 'Region/Regional Scope'
df[Region_column_name] = df[Region_column_name].apply(normalize_name)
df[Region_column_name] =df[Region_column_name].apply(lambda x: mapping(x))

df_ambig.to_csv("ambig.csv", index=False)
df_ambig.to_csv("./%s/AMBIG_%s.csv" %(folder_path, folder_path), index=False)


/var/folders/7z/6g3hfctx56g77d9vf2r0bbx00000gn/T/ipykernel_30150/140740927.py:3: ICUWarning: Install 'pyicu' for better text transliteration.
  return normalize(country, latinize=True)


In [35]:
input("Go to ambig.csv and correct ambiguous inputs")

''

In [6]:
def split_ambig():
    ambig = pd.read_csv('./ambig.csv')
    if ambig.empty:
        return ambig, ambig
    corrected = ambig[ambig['Quarantine?'] != 'x']
    further_quarantined = ambig[ambig['Quarantine?'] == 'x']
    return corrected, further_quarantined

def remove_quarantined(df_first_clean, further_quarantined, corrected):
    overlap_values = further_quarantined['Input string'].unique()
    removed_rows_df = df_first_clean[df_first_clean['Region/Regional Scope'].isin(overlap_values)].copy()
    df_second_clean = df_first_clean[~df_first_clean['Region/Regional Scope'].isin(overlap_values)]
    
    merged_df = pd.merge(df_second_clean, corrected, left_on='Region/Regional Scope', right_on='Input string', how='left')
    merged_df.loc[merged_df['Input string'].notna(), 'Region/Regional Scope'] = merged_df['Mapped code']
    merged_df.drop(columns=['Mapped code', "Input string","Mapped name","Quarantine?"], inplace=True)
    
    file_path = "./ambig.csv"
    if os.path.exists(file_path):
        shutil.move(file_path, "./%s/ambig_VERIFIED_%s.csv" % (folder_path, folder_path))
    return merged_df, removed_rows_df



corrected, further_quarantined = split_ambig()
merged_df, removed_rows_df =remove_quarantined(df, further_quarantined, corrected)




In [7]:
# In this cell, please add any code you'd like to edit 
print(merged_df['Region/Regional Scope'].unique())
print(removed_rows_df['Region/Regional Scope'].unique())


['MT' 'MM' 'MN' 'MZ' 'MR' 'MQ' 'MU' 'MW' 'MY' 'NA' 'NC' 'NE' 'GN' 'GP'
 'GM' 'GW' 'GQ' 'GR' 'GD' 'GL' 'GT' 'GF' 'GY' 'HK' 'HN' 'SC' 'SY' 'TC'
 'TD' 'TG' 'TH' 'TJ' 'TM' 'TL' 'TO' 'TT' 'TN' 'NG' 'NI' 'NL' 'SL' 'SV'
 'SO' 'PM' 'ST' 'SR' 'SK' 'SI' 'SE' 'SZ' 'PG' 'PL' 'PR' 'PT' 'PY' 'PF'
 'QA' 'RE' 'RO' 'RU' 'RW' 'SA' 'CSXX' 'SD' 'PH' 'SN' 'SG' 'SH' 'SB' 'MD'
 'MG' 'MV' 'MX' 'MK' 'ML' 'NO' 'NP' 'NZ' 'OM' 'PK' 'PA' 'PE' 'PW' 'LR'
 'IAI Area, Africa' 'LC' 'LK' 'LS' 'LT' 'LU' 'LV' 'MO' 'MA' 'GB' 'GE' 'GH'
 'GI' 'BW' 'CF' 'CA' 'CH' 'CL' 'CN' 'CI' 'CM' 'CD' 'CG' 'DJ' 'DM' 'DK'
 'DO' 'DZ' 'EC' 'EG' 'ER' 'EH' 'ES' 'CK' 'CO' 'KM' 'CV' 'CR' 'CU' 'KY'
 'CY' 'CZ' 'DE' 'EE' 'ET' 'FI' 'FJ' 'FK' 'FR' 'FO' 'GA' 'IN' 'IE' 'IR'
 'IQ' 'IS' 'IL' 'IT' 'JM' 'JO' 'JP' 'BS' 'BA' 'BY' 'BZ' 'BM' 'BO' 'BR'
 'BB' 'BN' 'BT' 'AT' 'AZ' 'BI' 'BE' 'BJ' 'BF' 'BD' 'BG' 'BH' 'HR' 'HT'
 'HU' 'ID' 'VC' 'VE' 'VG' 'VN' 'VU' 'WS' 'YE' 'ZA' 'ZM' 'ZW' 'AW' 'AF'
 'AO' 'AI' 'AL' 'ANHH' 'AE' 'AR' 'AM' 'AG' 'AU' 'KP' 'TR' 'TW' 'TZ' 'UG

In [8]:
input("Look through each dataset and adjust any inconsistent values")




''

In [9]:
removed_rows_df.to_csv("./%s/%s_further_quarantined.csv" %(folder_path, folder_path), index=False) 
merged_df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)


In [10]:
shutil.copy("./Nationalizer.ipynb", "%s" % folder_path)
shutil.move(csv_to_fix, "%s" % folder_path)
shutil.make_archive(folder_path.split('.')[0], 'zip', folder_path)
shutil.rmtree(folder_path)
